In [1]:
#GDD calculation for Soy from class Soy GDD. 
import pandas as pd

def fahrenheit_to_celsius(temp_f):
    return (temp_f - 32) * 5 / 9

def celsius_to_fahrenheit(temp_c):
    return (temp_c * 9 / 5) + 32

class SoyGDD:
    def __init__(self, daily_temp_df, time_column, min_temp_column, max_temp_column, celsius=True):
        self.daily_temp_df = daily_temp_df
        self.time_column = time_column
        self.min_temp_column = min_temp_column
        self.max_temp_column = max_temp_column
        self.celsius = celsius

        # Ensure the time column is of datetime type
        self.daily_temp_df[self.time_column] = pd.to_datetime(self.daily_temp_df[self.time_column])

        # If initial data is in Celsius, optionally convert it to Fahrenheit for uniform GDD calculation
        if self.celsius:
            self.daily_temp_df[self.min_temp_column] = self.daily_temp_df[self.min_temp_column].apply(celsius_to_fahrenheit)
            self.daily_temp_df[self.max_temp_column] = self.daily_temp_df[self.max_temp_column].apply(celsius_to_fahrenheit)
            self.celsius = False  # Now all data is in Fahrenheit

    def soy_gdd(self, start_date, end_date):
        start_date = pd.Timestamp(start_date)
        end_date = pd.Timestamp(end_date)

        date_mask = (self.daily_temp_df[self.time_column] >= start_date) & (self.daily_temp_df[self.time_column] <= end_date)
        masked_data = self.daily_temp_df.loc[date_mask].copy()

        gdd_values = []
        cumulative_gdd = 0

        for index, row in masked_data.iterrows():
            min_temp = max(row[self.min_temp_column], 50)  # Fahrenheit base
            max_temp = min(row[self.max_temp_column], 86)  # Fahrenheit upper limit

            mean_temp = (min_temp + max_temp) / 2
            base_temp = 50  # Fahrenheit base temperature for GDD calculation
            gdd = max(mean_temp - base_temp, 0)

            gdd_values.append(gdd)
            cumulative_gdd += gdd

        masked_data['GDD'] = gdd_values
        masked_data['Cumulative_GDD'] = masked_data['GDD'].cumsum()

        return masked_data

# This modified class converts temperatures to Fahrenheit if they start in Celsius, then computes and returns GDD in Fahrenheit.


import csv file with Min/Max temperature for Becker farm

In [6]:
import pandas as pd
Becker_GEMS = pd.read_csv("becker_gems_2023_2024.csv", on_bad_lines='skip')

In [7]:
Becker_GEMS_2024 = SoyGDD(Becker_GEMS, time_column='datetime', min_temp_column='min_temp', max_temp_column='max_temp')

In [8]:
#Using year 2024 start date and end date for mine gdd
start_date = pd.to_datetime("5-8-2024").date()
end_date = pd.to_datetime("9-17-2024").date()

Calculating Soy AGDD Becker 2024

In [9]:
Soy_gdd_BeckerGEMS2024 = Becker_GEMS_2024.soy_gdd(start_date, end_date)
print(Soy_gdd_BeckerGEMS2024)

     clouds   datetime  dewpt  dhi  dni  ghi  max_dhi  max_dni  max_ghi  \
493      11 2024-05-08    3.8   54  438  326      120      918      923   
494       7 2024-05-09    2.4   54  440  327      120      918      925   
495      21 2024-05-10    2.6   55  441  329      120      919      927   
496       0 2024-05-11    1.4   55  443  330      120      919      929   
497       0 2024-05-12    4.1   55  444  332      120      919      931   
..      ...        ...    ...  ...  ...  ...      ...      ...      ...   
621       0 2024-09-13   14.1   44  363  236      112      882      761   
622      69 2024-09-14   16.9   44  360  234      111      881      756   
623      41 2024-09-15   18.4   43  358  231      111      880      751   
624       9 2024-09-16   17.4   43  356  229      111      879      746   
625       2 2024-09-17   16.1   43  354  226      111      878      741   

     max_temp  ...  t_solar_rad  temp          ts  wind_dir  wind_gust_spd  \
493     74.66  ...   